In [1]:
import torch
import torchvision
import construct
import prep
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
image_dir = 'data/raman_images'
labels_file = 'data/labels.csv'

DATA = prep.prep_data(pd.read_csv(labels_file), image_dir)

transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = False
/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = True


In [3]:
class myModel(torch.nn.Module):
    def __init__(self):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
                
        #Convultion layer 2. See above
        self.conv2 = torch.nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5)
        
        self.fc_1 = torch.nn.Linear(39 * 39 * 12, 256)
        self.fc_2 = torch.nn.Linear(256, 2)
        self.drop = torch.nn.Dropout(p=.2)
        self.batch1 = torch.nn.BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch2 = torch.nn.BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            
    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """
        
        x = self.conv1(x)
        x = self.batch1(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 2)
        x = torch.nn.functional.leaky_relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 4)
        x = torch.nn.functional.leaky_relu(x)
        x = x.view(x.shape[0], -1)  
        x = self.fc_1(x) 
        x = torch.nn.functional.leaky_relu(x)
        x = self.drop(x)
        x = self.fc_2(x) 
        return x

model = myModel()

In [ ]:
#Declaring iterator. The thing that will loop through our dataset.

data = prep.tenX_dataset(DATA, 'data/raman_images', transform=transforms)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test, train = torch.utils.data.random_split(data, [int(.15*len(data)), len(data)-int(.15*len(data))])
BATCH_SIZE = 32
epochs = 150

In [ ]:
cnn, loss, acc = construct.train(epochs, BATCH_SIZE, train, criterion, 'Adam', model)

In [ ]:
images, labels, predictions, weights, acc = construct.get_predictions(BATCH_SIZE, cnn, test)

In [4]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
models = []
losses = []
train_accs = []
test_accs = []
naive_accs = []
epochs = 150
BATCH_SIZE = 32
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

for train_idx, test_idx in kf.split(DATA):
    curr_model = myModel()
    train = DATA.iloc[train_idx].reset_index()
    test = DATA.iloc[test_idx].reset_index()
    train_data = prep.tenX_dataset(train, 'data/raman_images', transform=transforms)
    test_data = prep.tenX_dataset(test, 'data/raman_images', transform=transforms)
    cnn, train_loss, train_acc = construct.train(epochs, BATCH_SIZE, 
                                                 train_data, criterion, 'Adam', curr_model, device)
    models.append(cnn)
    train_accs.append(train_acc)
    losses.append(train_loss)
    images, labels, predictions, weights, test_acc = construct.get_predictions(BATCH_SIZE, cnn, test_data)
    test_accs.append(test_acc)
    naive_accs.append((labels[:,1] == 0).float().sum()/len(predictions))
    

EPOCH: 0, acc: 0.785256408728086, loss: 3.0775509293262777
tensor([[ 2.2946,  3.3719],
        [ 0.7514, -4.7446],
        [ 7.8281, 12.9705],
        [ 3.9225,  2.0446],
        [ 5.3197,  2.0035],
        [ 6.3861,  6.2785],
        [ 1.0495,  3.5111],
        [27.2679, 15.3362],
        [-0.4809,  1.1625],
        [ 5.3535, -9.9138],
        [ 3.0353,  5.3463],
        [-4.7729, -8.6513],
        [ 3.9222,  2.7027],
        [ 4.7894,  1.0953],
        [ 7.9600,  3.4941],
        [ 1.7643, -0.0438],
        [ 6.4122, -1.9505],
        [ 3.6090,  2.0332],
        [ 0.2661, -0.1384],
        [ 3.1906,  2.4718],
        [ 2.2868, -2.2342],
        [ 3.6915,  0.6390],
        [ 3.7347,  7.8855],
        [13.8402, 21.4532]], grad_fn=<AddmmBackward>)
tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
   

KeyboardInterrupt: 

In [ ]:
for i in losses:
    plt.plot(i)

In [ ]:
test_accs

In [ ]:
naive_accs

In [ ]:
(labels[:,1] == predictions).float().sum()/len(predictions)

In [ ]:
(labels[:,1] == 0).float().sum()/len(predictions)

In [ ]:
def test_dataset_class():
    filename = '../tests/test_assets/test_labels_unclean.csv'
    image_dir = '../tests/test_assets/test_images'
    
    #Eventually just read in already cleaned labels sheet
    labels = prep_data(pd.read_csv(filename), image_dir)
    
    
    transform = None
    test_dataset = tenX_dataset(labels, image_dir, transform)
    
    #len() check
    length = len(test_dataset)
    expect = 5
    assert length is expect, f'10x dataset length method failed. Got {length}, should be {expect}'
    
    #get_item() check
    samples = []
    keys = ['image', 'plastic','shape','color']
    for i in range(len(test_dataset)):
        assert test_dataset[i]['image'] is not None, 'Got NoneType instead of image'
        isP = test_dataset[i]['plastic']
        assert math.isclose(0,isP) or math.isclose(1,isP), f'plastic not 0 or 1, instead is {isP}'
        length = len(test_dataset[i]['shape'])
        assert length is 4, f'length of shape array not 4, instead is {length}'
        samples.append(test_dataset[i])
        
    shape = samples[0]['shape']
    assert math.isclose(shape[2], 1),  f'wrong shape first image, is {shape}'
    color = samples[0]['color']
    assert math.isclose(color[0], 1), f'wrong color first image, is {color}'
    assert math.isclose(samples[4]['plastic'], 1), 'wrong plastic id 3rd image'
    
test_dataset_class()

In [ ]:
#0 - BASE LINE 82% naive and 88% model test set accuracy
#1 - 9 output CNN1, 18 output CNN2 87% test vs 77% naive
#2 - 3rd linear layer telescoped (12*39*39 -> 2048 -> 256- >2). 93% test, 87% naive
#3 - added a convulutional layer, 12 -> 24 no good
#2d dropout after convolutions led to predicting all pastics, i also read dropouts after convultions is bad, going to try batch normalizations though
#Another paper I read said that non-adaptive optimizers were better for CNN's if thehyperparameters are tunned
#tuned correctly but are much slower so I don't want to even try given it takes me 20min to train already
#adding batchnormalization after convultions improved training results alot, but not test which is weird since
#its supposed to reduce overfitting. Im going to run again overnight